# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [28]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import time
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [8]:
#load csv into dataframe
csv_path = "../cities.csv"
city_weather_df = pd.read_csv(r"../output_data/cities.csv")

#drop rows with missing data
city_weather_df = city_weather_df.dropna()

city_weather_df


,Unnamed: 0,City,Latitude,Longitude,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,Rikitea,-23.1203,-134.9692,78.89,68,6,13.62,PF,1650324679
1,1,Cape Town,-33.9258,18.4232,59.92,88,0,13.00,ZA,1650324593
2,2,Vao,-22.6667,167.4833,75.99,75,50,5.44,NC,1650324866
3,3,Rawson,-43.3002,-65.1023,61.23,67,5,5.17,AR,1650324866
4,4,Mareeba,-17.0000,145.4333,77.67,77,100,4.88,AU,1650324867
...,...,...,...,...,...,...,...,...,...,...
552,552,Anzhero-Sudzhensk,56.0810,86.0285,28.74,78,74,4.99,RU,1650325200
553,553,Francistown,-21.1700,27.5079,60.96,80,55,5.79,BW,1650325200
554,554,Grogol,-7.6011,110.8186,78.24,88,100,1.21,ID,1650325201
555,555,Lethem,3.3803,-59.7968,83.62,56,67,9.33,GY,1650325201


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [9]:
#configure gmaps with API key 
gmaps.configure(g_key)

locations = city_weather_df[["Latitude", "Longitude"]]
humidity = city_weather_df["Humidity"].astype(float)

In [10]:
fig = gmaps.figure()

heat_layer = gmaps.heatmap_layer(locations, weights=humidity,
                                dissipating=False, max_intensity=10,
                                point_radius=1)
fig.add_layer(heat_layer)

fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [31]:
# Narrow down the cities with wind speed less than 10 mph, cloudiness equals to 0 and max temp between 60 and 80
narrowed_city_df = city_weather_df.loc[(city_weather_df["Wind Speed"] <= 10) & (city_weather_df["Cloudiness"] == 0) & \
                                   (city_weather_df["Max Temp"] >= 70) & (city_weather_df["Max Temp"] <= 80)].dropna()

narrowed_city_df

,Unnamed: 0,City,Latitude,Longitude,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
172,172,Vredendal,-31.6683,18.5012,72.79,24,0,5.70,ZA,1650324947
280,280,Saint-Pierre,-21.3393,55.4781,76.91,80,0,3.44,RE,1650324995
362,362,Casino,-28.8667,153.0500,71.78,83,0,4.61,AU,1650325107


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [40]:
# Create a hotel_df
hotel_df = narrowed_city_df.loc[:,["City","Country", "Latitude", "Longitude"]]

# Add a "Hotel Name" column to the DataFrame.
hotel_df["Hotel Name"] = ""

# Display the result
hotel_df

,City,Country,Latitude,Longitude,Hotel Name
172,Vredendal,ZA,-31.6683,18.5012,
280,Saint-Pierre,RE,-21.3393,55.4781,
362,Casino,AU,-28.8667,153.0500,


In [41]:
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

params = {"type" : "hotel",
          "keyword" : "hotel",
          "radius" : 5000,
          "key" : g_key}

In [42]:
for index, row in hotel_df.iterrows():
    # get city name, lat, lnt from df
    lat = row["Latitude"]
    lng = row["Longitude"]
    city_name = row["City"]
    
    # add keyword to params dict
    params["location"] = f"{lat},{lng}"

    # assemble url and make API request
    print(f"Retrieving Results for Index {index}: {city_name}.")
    response = requests.get(base_url, params=params).json()
    
    # extract results
    results = response['results']
    
    # save the hotel name to dataframe
    try:
        print(f"Closest hotel in {city_name} is {results[0]['name']}.")
        hotel_df.loc[index, "Hotel Name"] = results[0]['name']

    # if there is no hotel available, show missing field
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")
        
    print("------------")
    
    # Wait 1 sec to make another api request to avoid SSL Error
    time.sleep(1)

# Print end of search once searching is completed
print("-------End of Search-------")

Retrieving Results for Index 172: Vredendal.
Closest hotel in Vredendal is Tharrakamma Guest House.
------------
Retrieving Results for Index 280: Saint-Pierre.
Closest hotel in Saint-Pierre is Hôtel Le Saint-Pierre.
------------
Retrieving Results for Index 362: Casino.
Closest hotel in Casino is Hotel Cecil.
------------
-------End of Search-------


In [43]:
hotel_df

,City,Country,Latitude,Longitude,Hotel Name
172,Vredendal,ZA,-31.6683,18.5012,Tharrakamma Guest House
280,Saint-Pierre,RE,-21.3393,55.4781,Hôtel Le Saint-Pierre
362,Casino,AU,-28.8667,153.0500,Hotel Cecil


In [44]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Latitude", "Longitude"]]

In [45]:
# Add marker layer and info box content ontop of heat map
markers = gmaps.marker_layer(locations, info_box_content = hotel_info)

# Add the layer to the map
fig.add_layer(markers)

# Display Map
fig

Figure(layout=FigureLayout(height='420px'))